# LAB - Luminance, a (green-red), b (blue-yellow) colorspace
Stough, DIP

- [skimage's color module](https://scikit-image.org/docs/dev/api/skimage.color.html)
- [What is L*a*b*?](https://en.wikipedia.org/wiki/CIELAB_color_space)
- [Why Lab?](https://graphicdesign.stackexchange.com/questions/76824/what-are-the-pros-and-cons-of-using-lab-color)

The Lab colorspace is also oriented around adjusting color according to human vision
*perception* as opposed to mechanism (r-g-b stimulus).

<img src="https://i.pinimg.com/originals/bf/2d/0e/bf2d0e090f386111cae1d8e06d4f0015.jpg" style="width:auto;height:200px"/> <img src="https://www.color-management-guide.com/images/basics-of-color/chromacity-diagram-lab-space.jpg" style="width:auto;height:200px"/> 

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt
import numpy as np
import skimage.color as color
import matplotlib.colors as mcolors

# For importing from alternative directory sources
import sys  
sys.path.insert(0, '../dip_utils')

import matrix_utils
from vis_utils import (vis_lab_cube,
                       vis_hists,
                       lab_uniform)

In [ ]:
IMAGE = 'mountainSpring.jpg'
# IMAGE = 'peppers.png'

# Let's construct our own colormaps
cm_KR = mcolors.LinearSegmentedColormap.from_list('kr', [(0,0,0), (1, 0, 0)], 256)
cm_KG = mcolors.LinearSegmentedColormap.from_list('kg', [(0,0,0), (0, 1, 0)], 256)
cm_KB = mcolors.LinearSegmentedColormap.from_list('kb', [(0,0,0), (0, 0, 1)], 256)

In [ ]:
Irgb = plt.imread('../dip_pics/' + IMAGE)

# Image of uniform random distributed color
# Irgb = np.random.random((100,100,3))

In [ ]:
# Let's construct our own colormaps for the a and b dimensions.
colorsA = [(0,1,0), (.5,.5,.5), (1,0,0)] # green -> red
colorsB = [(0,0,1), (.5,.5,.5), (1,1,0)] # blue -> yellow

cmA = mcolors.LinearSegmentedColormap.from_list('GreenRed', lab_uniform(colorsA), 256)
cmB = mcolors.LinearSegmentedColormap.from_list('BlueYellow', lab_uniform(colorsB), 256)

In [ ]:
f, ax = plt.subplots(3,3, figsize = (8,6), sharex=True, sharey=True)

Ilab = color.rgb2lab(Irgb)

# Row 0: just the full color image
ax[0][1].imshow(Irgb)
ax[0][0].set_axis_off()
ax[0][2].set_axis_off()

# Row 1: r g b channels
for i in range(3):
    ax[1,i].imshow(Irgb[...,i], cmap=[cm_KR, cm_KG, cm_KB][i])
    ax[1,i].set_title(['Red', 'Green', 'Blue'][i])


# Now to display the HSV space.

ax[2,0].imshow(Ilab[...,0], cmap='gray')
ax[2,0].set_title('L')

ax[2,1].imshow(Ilab[...,1], cmap=cmA)
ax[2,1].set_title('a - green->red')

ax[2,2].imshow(Ilab[...,2], cmap=cmB)
ax[2,2].set_title('b - blue->yellow')

plt.tight_layout()
f.canvas.set_window_title('RGB vs HSV')
plt.show()

&nbsp;

## Let's visualize in 3D

In [ ]:
vis_lab_cube(Irgb)

In [ ]:
vis_lab_cube(np.random.random((100,100,3)))